# Обучение ML модели для извлечения полей

Этот ноутбук демонстрирует процесс обучения модели для извлечения полей из платежных счетов.


## Импорт библиотек


In [ ]:
import sys
from pathlib import Path
import json
from io import BytesIO

# Добавляем путь к backend
sys.path.append(str(Path('../backend').absolute()))

from app.models.ml_model import FieldExtractionModel
from app.services.extractor import PaymentInvoiceExtractor
from app.utils.pdf_parser import PDFParser


## Подготовка данных для обучения


In [ ]:
# Примеры данных для обучения
# В реальности эти данные должны быть аннотированы экспертами

training_examples = [
    {
        "text": "Плательщик: ООО Пример ИНН: 1234567890 КПП: 123456789 Сумма: 10000.00",
        "expected_fields": {
            "плательщик": {"наименование": "ООО Пример", "ИНН": "1234567890", "КПП": "123456789"},
            "сумма": "10000.00"
        }
    },
    # Добавьте больше примеров здесь
]

print(f"Подготовлено {len(training_examples)} примеров для обучения")


## Обучение модели


In [ ]:
# Инициализация модели
model = FieldExtractionModel()

# Подготовка данных
training_data = [
    (example["text"], example["expected_fields"])
    for example in training_examples
]

# Обучение
if training_data:
    model.train(training_data)
    print("✓ Модель успешно обучена")
else:
    print("⚠ Нет данных для обучения")


## Тестирование модели


In [ ]:
# Тестовый текст
test_text = """
Плательщик: ООО Тестовая Компания
ИНН: 9876543210
КПП: 987654321
Получатель: ООО Получатель
ИНН: 1111111111
Сумма: 50000.00 руб.
Назначение платежа: Оплата по договору №123
"""

# Извлечение полей
result = model.extract_fields(test_text)

print("Результат извлечения:")
print(json.dumps(result, ensure_ascii=False, indent=2))
